## Colmap 3D Reconstruction
This notebook provides steps on running Colmap Sparse Reconstruction in Colab

Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Colmap

In [ ]:
!apt-get update
!apt-get install -y colmap ffmpeg
!nvidia-smi
!colmap -h

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,063 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

Import libraries

In [ ]:
import cv2
import numpy as np
import os
import shutil

Preprocess the frames (those particular settings worked well with the 10SOM282 video)

In [ ]:
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, h=10)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)
    gamma = 1.2
    lut = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(256)]).astype("uint8")
    gamma_corrected = cv2.LUT(enhanced, lut)
    edges = cv2.Laplacian(gamma_corrected, cv2.CV_64F)
    edges = cv2.convertScaleAbs(edges)
    enhanced_edges = cv2.addWeighted(gamma_corrected, 0.8, edges, 0.2, 0)
    sharpening_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(enhanced_edges, -1, sharpening_kernel)
    return sharpened

def process_video_fragment(video_path, output_folder, start_time, end_time, sample_fps=10, fragment_number=1):
    cap = cv2.VideoCapture(video_path)
    actual_fps = cap.get(cv2.CAP_PROP_FPS)
    os.makedirs(output_folder, exist_ok=True)

    print(f"\nProcessing fragment: {start_time}s to {end_time}s")
    start_frame = int(start_time * actual_fps)
    end_frame = int(end_time * actual_fps)
    frame_interval = max(1, int(actual_fps / sample_fps))

    frag_folder = os.path.join(output_folder, f"fragment_{fragment_number}")
    os.makedirs(frag_folder, exist_ok=True)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_idx = start_frame
    saved_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_idx > end_frame:
            break

        if frame_idx % frame_interval == 0:
            processed = preprocess_frame(frame)
            base_fname = f"{frag_folder}/frame_{saved_idx:04d}"
            cv2.imwrite(f"{base_fname}.png", processed)
            saved_idx += 1
        frame_idx += 1

    cap.release()

# Define the input and output directories
video_file = "/content/drive/MyDrive/10SOM282.mp4"
output_dir = "/content/drive/MyDrive/processed_frames"

# Define the time constraints of the segment you want to reconstruct
start_time = 9 * 60 + 21
end_time = 9 * 60 + 42
fragment_number = 1

# Process the video fragment
process_video_fragment(video_file, output_dir, start_time, end_time, fragment_number)



Processing fragment: 561s to 582s


Set up the COLMAP directory structure and copy the preprocessed files

In [ ]:
# Set up COLMAP project structure
project_dir = "/content/drive/MyDrive/colmap_project"
images_dir = os.path.join(project_dir, "images")
sparse_dir = os.path.join(project_dir, "sparse")
dense_dir = os.path.join(project_dir, "dense")

os.makedirs(images_dir, exist_ok=True)
os.makedirs(sparse_dir, exist_ok=True)
os.makedirs(dense_dir, exist_ok=True)

# Source directory
source_dir = f"/content/drive/MyDrive/processed_frames/fragment_{fragment_number}"

# List and sort files
all_files = sorted(os.listdir(source_dir))

# Copy selected files
for filename in all_files:
    src = os.path.join(source_dir, filename)
    dst = os.path.join(images_dir, filename)
    if os.path.isfile(src):
        shutil.copy2(src, dst)


Extract the features from the frames with SIFT

In [ ]:
!colmap feature_extractor \
    --database_path /content/drive/MyDrive/colmap_project/database.db \
    --image_path /content/drive/MyDrive/colmap_project/images \
    --ImageReader.single_camera 1 \
    --ImageReader.camera_model PINHOLE \
    --SiftExtraction.use_gpu 0


Feature extraction

Processed file [1/315]
  Name:            frame_0000.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PINHOLE
  Focal Length:    2304.00px
  Features:        7310
Processed file [2/315]
  Name:            frame_0001.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PINHOLE
  Focal Length:    2304.00px
  Features:        6738
Processed file [3/315]
  Name:            frame_0002.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PINHOLE
  Focal Length:    2304.00px
  Features:        5596
Processed file [4/315]
  Name:            frame_0003.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PINHOLE
  Focal Length:    2304.00px
  Features:        5890
Processed file [5/315]
  Name:            frame_0004.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PINHOLE
  Focal Length:    2304.00px
  Features:        6385
Processed file [6/315]
  Name:            frame_0005.png
  Dimensions:      1920 x 1080
  Camera:          #1 - PI

Match the features using the sequential matcher

In [ ]:
!colmap sequential_matcher \
    --database_path /content/drive/MyDrive/colmap_project/database.db \
    --SiftMatching.use_gpu 0


Sequential feature matching

Matching image [1/315] in 13.834s
Matching image [2/315] in 13.424s
Matching image [3/315] in 12.394s
Matching image [4/315] in 12.993s
Matching image [5/315] in 13.715s
Matching image [6/315] in 14.196s
Matching image [7/315] in 14.830s
Matching image [8/315] in 14.621s
Matching image [9/315] in 15.094s
Matching image [10/315] in 16.579s
Matching image [11/315] in 23.574s
Matching image [12/315] in 19.794s
Matching image [13/315] in 18.416s
Matching image [14/315] in 17.710s
Matching image [15/315] in 17.982s
Matching image [16/315] in 18.746s
Matching image [17/315] in 17.888s
Matching image [18/315] in 17.259s
Matching image [19/315] in 13.207s
Matching image [20/315] in 13.987s
Matching image [21/315] in 15.185s
Matching image [22/315] in 16.068s
Matching image [23/315] in 17.857s
Matching image [24/315] in 18.260s
Matching image [25/315] in 20.923s
Matching image [26/315] in 19.354s
Matching image [27/315] in 18.929s
Matching image [28/315] in 22.294s

Run sparse reconstruction (mapping)

In [ ]:
!colmap mapper \
    --database_path /content/drive/MyDrive/colmap_project/database.db \
    --image_path /content/drive/MyDrive/colmap_project/images \
    --output_path /content/drive/MyDrive/colmap_project/sparse \
    --Mapper.init_max_forward_motion 1 \
    --Mapper.init_min_tri_angle 0.5

Streaming output truncated to the last 5000 lines.
    Residuals : 31864
   Parameters : 3173
   Iterations : 4
         Time : 0.243076 [s]
 Initial cost : 0.778228 [px]
   Final cost : 0.771165 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 39
  => Filtered observations: 7
  => Changed observations: 0.002438

Registering image #234 (234)

  => Image sees 1238 / 4744 points

Pose refinement report
----------------------
    Residuals : 2488
   Parameters : 6
   Iterations : 8
         Time : 0.0242102 [s]
 Initial cost : 0.650271 [px]
   Final cost : 0.648036 [px]
  Termination : Convergence

  => Continued observations: 1232
  => Added observations: 1051

Bundle adjustment report
------------------------
    Residuals : 29300
   Parameters : 3272
   Iterations : 18
         Time : 0.976402 [s]
 Initial cost : 0.629572 [px]
   Final cost : 0.620135 [px]
  Termination : Convergence

  => Merged observations: 49
  => Completed observations: 76

Convert the sparse reconstruction to text format (camera poses and 3D points)

In [ ]:
!mkdir -p /content/drive/MyDrive/colmap_project/sparse_text
!colmap model_converter \
    --input_path /content/drive/MyDrive/colmap_project/sparse/0 \
    --output_path /content/drive/MyDrive/colmap_project/sparse_text \
    --output_type TXT